In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
RL='https://devimpactinstitute.com/'

In [3]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [4]:
driver.get(RL)

In [5]:
soupn = bs(driver.page_source,'lxml')
hres=soupn.find('div',{'class':'slick-track'}).find_all('a')
urls=[]
url=[a['href'] for a in hres]
[urls.append(a) for a in url if a not in urls]
urls

['https://devimpactinstitute.com/courses/project-management',
 'https://devimpactinstitute.com/courses/gis',
 'https://devimpactinstitute.com/courses/mobile-technologies',
 'https://devimpactinstitute.com/courses/agriculture-climate-change-and-rural-development',
 'https://devimpactinstitute.com/courses/research-monitoring-and-evaluation',
 'https://devimpactinstitute.com/courses/data-management-and-statistical-analysis',
 'https://devimpactinstitute.com/courses/business']

In [6]:
div = 'main-listing'
item = 'listing row' #div

In [7]:
#get the title and link from the first oage
data = []
for i in urls:
    driver.get(i)
    time.sleep(10)
    try:
        soup1 = bs(driver.page_source,'lxml')
        container = soup1.find('div',{'class':div})
        each_item = container.find_all('div',{'class':item})
        for j in each_item:
            data.append([j.h4.text.strip(),j.a['href']])
    except:
        pass

In [17]:
#loop through the link and extract the two tables
devisti = []
placeholder1=''
for l in data:
    driver.get(l[-1])
    time.sleep(5)
    soup = bs(driver.page_source,'lxml')
    maio=soup.find('section').find('div',{'class':'description'})
    dati= maio.findChild().next_siblings
    quak=[bap for bap in dati]
    try:
        while True:
            quak.remove('\n')
    except ValueError:
        pass
    if '<ul>' in str(quak[1]):
        info=(quak[0].text+quak[1].text).replace(':\n',': ').replace('\n','• ').replace('\xa0\xa0',' ').replace('\xa0',' ')
    else:
        info=quak[0].text.replace('\xa0\xa0',' ').replace('\xa0',' ')
    tables = soup.find_all('table',{'class':'table table-responsive'})
    table_data = []
    topic = l[0]
    mass=soup.find('ul',{'id':'top-bar'}).find_all('li')[-1].text.split(':')[-1].strip()
    #there are three tables on the page, we are interested in the last two (virtual and classroom)
    #virtual has number of days, clasroom has location
    
    
    #**online**
    #for g in tables[1:]:
    for g in tables[:1]:
        body = g.tbody
        tr = body.find_all('tr')
        for j in tr:
            spar=j.find_all('td')
            onof='1'
            if onof=='1':
                loc=''
            td = [topic] +[k.text.strip() for k in [spar[0],spar[1],spar[3]]] + [j.a['href']]+[onof]+[loc]
            table_data.append(td)
    for h in tables[1:]:
        body = h.tbody
        ti = body.find_all('tr')
        for r in ti:
            spu=r.find_all('td')
            onof='0'
            tm = [topic] +[k.text.strip() for k in [spu[0],spu[1],spu[4]]] + [r.a['href']]+[onof]+[k.text.strip() for k in [spu[2]]]
            table_data.append(tm)
    
    for sas in table_data:
        title=sas[0]
        link=l[1]
        stuar=[sas[1],sas[2]]
        spl_dt_obj = [datetime.strptime(v, '%B %d %Y') for v in stuar]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
        price=sas[3].replace(',','')
        if price=='':
            ticketlist=''
        else:
            if int(price)>0:
                ty='paid'
                am=price
                cu='$'
            else:
                ty='free'
                am=''
                cu=''
            ticketlist={'type': ty,
                         'price':am,
                         'currency': cu}
        if ticketlist=='':
            ticket_list=''
        else:
            ticket_list=json.dumps(ticketlist)
            
        if sas[4]=='':
            event_web=link
        else:
            event_web=sas[4]
            
         ###############COUNTYR,GPU#################
        def countr(locale):
            try:
                google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                try:
                    google_map_url=driver.find_element_by_id('lu_map').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                    except:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                time.sleep(1)
                try:
                    country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
                except:
                    country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
                return(country)
            except Exception as e:
                print(e)
                return("")

        ####################################
        def get_google_map_url(location):
            try:
                google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
                time.sleep(randint(0,3))
                driver.get(google_url_for_location)
                time.sleep(4)
                try:
                    google_map_url=driver.find_element_by_id('lu_map').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                    except:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                time.sleep(1)
                google_map_url=driver.current_url
#                 print(google_map_url)
                return(google_map_url)
            except Exception as e:
                print(e)
                return("")
            
            
        if sas[5]=='1':
            on_off='1'
            city=''
            venue=''
            country=''
            googlePlaceUrl=''
        elif sas[5]=='0':
            on_off='0'
            city=sas[6]
            venue=sas[6]
            #avoid running maps several times
            if city==placeholder1:
                country=country
            else:
                country=countr(city)
            
            
            if venue==placeholder1:
                googlePlaceUrl=googlePlaceUrl
            else:
                googlePlaceUrl=get_google_map_url(venue)
            placeholder1=sas[6]
            #placeholder1=sas[6]
            #country=country(city)
#             for idx,o in enumerate(devisti):
#                 if city==o[15]:
#                     country=o[16]
#                 else:
#                     country=country(city)
#             for idx,o in enumerate(devisti):
#                 if venue==o[17]:
#                     googlePlaceUrl=o[19]
#                 else:
#                     googlePlaceUrl=get_google_map_url(venue)
            
        else:
            on_off='1'
            city=''
            venue=''
            country=''
            googlePlaceUrl=''
        mail_=json.dumps(mass)
        time_=''
        event_info=info
        
        org_pro='Devimpact Institute is a leading provider of research, training and consultancy services to international development organizations. We assist our partners to increase the impact and efficiency of their social interventions by providing a range of innovative solutions for sustainable results.'
        org_name='Devimpact Institute'
        org_web='https://devimpactinstitute.com/'
        ticket_list=''
        logo=''
        sponsors=''
        agendalist=''
        type_=''
        category=''
        Speakerlist=''
        
        
        
        devisti.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsors,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])
        
print('Done')

Done


In [18]:
len(devisti)

1589

In [19]:
devisti

[['https://devimpactinstitute.com/courses/project-management/training-on-child-protection',
  'Training on Child Protection',
  '2022-05-09',
  '2022-05-17',
  '',
  "Child protection consists of reducing risks to children’s holistic wellbeing, making children's rights to protection a reality, restoring hope and a dignified life where abuse and violence has occured and creating an enabling environment that supports children's positive development. Child protection work considers systems and community-based approaches to protection, including change at the institutional, community, family and individual level.",
  '',
  'Devimpact Institute is a leading provider of research, training and consultancy services to international development organizations. We assist our partners to increase the impact and efficiency of their social interventions by providing a range of innovative solutions for sustainable results.',
  'Devimpact Institute',
  'https://devimpactinstitute.com/',
  '',
  '',
  

In [20]:
devisti_dict=devisti


devisti_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=devisti_dict)
devisti_df.to_csv("devisti.tsv", sep = '\t',index=False)
print('Dusted')

Dusted
